In [2]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
from SAC_Distillation.DistilledSACAgent import DistilledSAC
from SAC_Distillation.Trajectories import SAC_ExperienceBuffer
from Hyperparameters import HYPERPARAMS as params
import numpy as np
import torch
import wandb

In [3]:
def relocate_agents(env):
    return list(env.agents)  # simplified

# New helper to extract observation data for an agent
def get_agent_obs(obs, agent):
    agent_data = obs[agent]
    return np.array(agent_data[1]), np.array(agent_data[2])

In [4]:
env = UE(file_name="Env/DroneFlightv1", seed=1, no_graphics_monitor=True, no_graphics=True)
env = UPZBE(env)

In [5]:
agents = relocate_agents(env)
print(agents)

['Drone?team=0?agent_id=0', 'Drone?team=0?agent_id=1', 'Drone?team=0?agent_id=10', 'Drone?team=0?agent_id=11', 'Drone?team=0?agent_id=12', 'Drone?team=0?agent_id=13', 'Drone?team=0?agent_id=14', 'Drone?team=0?agent_id=15', 'Drone?team=0?agent_id=2', 'Drone?team=0?agent_id=3', 'Drone?team=0?agent_id=4', 'Drone?team=0?agent_id=5', 'Drone?team=0?agent_id=6', 'Drone?team=0?agent_id=7', 'Drone?team=0?agent_id=8', 'Drone?team=0?agent_id=9']


In [6]:
# obs = env.reset()
# print(obs[agents[0]][1])
# possible_actions = env.action_space(agents[0]).sample()
# print(f"Possible actions: {possible_actions}")
# print(env.action_space(agents[0]).shape)
# print(env.observation_space(agents[0])[1].shape)
# print(env.observation_space(agents[0])[2].shape)

In [7]:
Buffer = SAC_ExperienceBuffer(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape,env.action_space(agents[0]).shape, params['sac_distilled'])

In [8]:
brain = DistilledSAC(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape,len(agents), params['sac_distilled'])

In [9]:
for s in range(1, params['sac_distilled'].seed_episodes):
    obs, done, t = env.reset(), [False for _ in env.agents], 0
    while not all(done) or t < params['sac_distilled'].n_steps_random_exploration:
        actions = {}
        log_probs = {}
        values = {}
        agents = relocate_agents(env)
        for agent in agents:
            actions[agent] = env.action_space(agent).sample()
            if agent not in obs.keys():
                continue
            obs1, obs2 = get_agent_obs(obs, agent)
            v1,v2 = brain.get_values(obs1,obs2, actions[agent],t)
            values[agent] = torch.min(v1,v2)


        next_obs, reward, done, _ = env.step(actions)

        for agent in agents:
            if agent not in next_obs.keys():
                continue
            next_obs1, next_obs2 = get_agent_obs(next_obs, agent)
            Buffer.store(obs1, obs2, actions[agent], reward[agent], next_obs1, next_obs2, done[agent])
        obs = next_obs
        done = [done[agent] for agent in agents if agent in done.keys()]
        t += 1
    print(f'Finished episode {s}')

# Buffer.compute_advantages()
print("Finished Rnd Exploration")
env.close()

Finished episode 1
Finished episode 2
Finished Rnd Exploration


In [10]:
brain.fine_tune_teacher(Buffer, epochs=2)

Epoch 0 completed
Epoch 1 completed
Teacher model fine-tuning completed


In [11]:
brain.teacher.save("SavedModels/Teacher")

In [ ]:
brain.train(Buffer,step = params['sac_distilled'].seed_episodes*params['sac_distilled'].n_steps_random_exploration)

In [ ]:
Buffer._clear_old()

In [ ]:
torch.save(brain.net.state_dict(), "SavedModels/SAC_distilled_checkpoint.pth")
print("Checkpoint saved successfully.")

In [1]:
wandb.init(project="SAC_Distillation", entity="fede-")
wandb.config.update(params['sac_distilled'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.config.update({"device": device})

NameError: name 'wandb' is not defined

In [ ]:
env = UE(file_name="DroneFlightv1", seed=1, side_channels=[], no_graphics_monitor=True, no_graphics=True)
env = UPZBE(env)
agents = relocate_agents(env)

In [ ]:
steps = 0
while steps < params['sac_distilled'].max_steps:
    obs, done, t = env.reset(), [False for _ in env.agents], 0
    episode_reward = 0
    while not all(done) or t < params['sac_distilled'].n_steps:
        actions = {}
        log_probs = {}
        values = {}
        agents = relocate_agents(env)
        for agent in agents:
            actions[agent] = env.action_space(agent).sample()
            if agent not in obs.keys():
                continue
            obs1, obs2 = get_agent_obs(obs, agent)
            v1,v2 = brain.get_values(obs1,obs2, actions[agent], steps+t)
            values[agent] = torch.min(v1,v2)


        next_obs, reward, done, _ = env.step(actions)

        for agent in agents:
            if agent not in next_obs.keys():
                continue
            next_obs1, next_obs2 = get_agent_obs(next_obs, agent)
            Buffer.store(obs1, obs2, actions[agent], reward[agent], next_obs1, next_obs2, done[agent])
        obs = next_obs
        done = [done[agent] for agent in agents if agent in done.keys()]
        tot_reward = [reward[agent] for agent in agents if agent in reward.keys()]
        t += 1
        
    obs_keys = list(obs.keys())
    mean_reward = np.mean(tot_reward)
    steps += t
    

    # SAC optimization step
    brain.train(Buffer, steps)

    brain.actor_optimizer = brain.adjust_lr(brain.actor_optimizer, params['sac_distilled'].actor_lr, steps, params['sac_distilled'].n_steps)
    brain.critic_optim = brain.adjust_lr(brain.critic_optim, params['sac_distilled'].critic_lr, steps, params['sac_distilled'].n_steps)
    brain.distill_optimizer = brain.adjust_lr(brain.distill_optimizer, params['sac_distilled'].distill_lr, steps, params['sac_distilled'].n_steps)
    wandb.log({"Mean Reward": mean_reward, "Steps": steps})
env.close()

c:\Users\rullo\Desktop\Master_Thesis\Project\Dreamer\SAC_Distillation\DistilledSACAgent.py:186: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(q1, target_q) + F.mse_loss(q2, target_q)


AttributeError: 'types.SimpleNamespace' object has no attribute 'lr'

In [ ]:
env.close()

In [ ]:
torch.save(brain.net.state_dict(), "SavedModels/SAC_distilled_checkpoint.pth")
print("Checkpoint saved successfully.")

In [ ]:
# Ensure the model is in evaluation mode
brain.net.eval()

# Create dummy input matching the expected input format of the model
dummy_input_1 = torch.randn(1, *env.observation_space(agents[0])[1].shape).to(device)
dummy_input_2 = torch.randn(1, *env.observation_space(agents[0])[2].shape).to(device)

# Export the model to ONNX format
torch.onnx.export(
    brain.net,
    (dummy_input_1, dummy_input_2),
    "SavedModels/SAC_distilled.onnx",
    export_params=True,
    opset_version=10,
    do_constant_folding=True,
    input_names=["observation1", "observation2"],
    output_names=["action"],
)
print("Model exported to ONNX format successfully.")

# Dispose of the dummy input tensors
del dummy_input_1
del dummy_input_2
torch.cuda.empty_cache()